In [ ]:
import os
import torch

import numpy as np
import matplotlib.pyplot as plt

# A short example: transonic cylinder flow

## Dataset and Dataloader

In [ ]:
from pbdl.torch.loader import *
import examples.tcf.net_small as net_small

dataset = Dataset(
    "transonic-cylinder-flow-tiny", # dataset name
    time_steps=10, # time steps between input and target frame
    sel_sims=[0,1], # use only the first two simulations
    step_size=3, # trim_start=100, trim_end=100,
    normalize=True
)

loader = Dataloader(dataset, # batch_sampler=...,
    batch_size=3, shuffle=True)

net = net_small.NetworkSmall()
criterionL2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.0)

A PBDLDataset instance provides the data samples to the data loader and must be initialized with the dataset name and the time steps between input and target sample. Due to the specification of `sel_sims`, only the first two simulations are used. As the transonic-cylinder-flow dataset is quite large for quick training, it is thinned out using the parameter `step_size` (every third sample is used). Additionally, you could use `trim_start` and `trim_end` to discard a (possibly uninteresting) start/end sequence of samples. Optionally, it is possible to specify whether the data should be normalized.

A PBDLDataLoader instance takes the dataset as input and prepares the samples for later training with the CNN (e.g. blows up the constants into layers). A loader instance is iterable, returning batches of the specified `batch_size`. When integrating solvers into deep learning, it may be required for samples in a batch to have the same constants; this is ensured by the PBDLConstantBatchSampler (not used in this example).

## Training

In [ ]:
for epoch in range(5):
    for i, (input, target) in enumerate(loader):

        net.zero_grad()
        output = net(input)

        loss = criterionL2(output, target)
        loss.backward()
        optimizer.step()

    print(f"epoch { epoch }, loss { loss.item() }")

## Evaluation

In [ ]:
net.eval()

input, target = next(iter(loader))
output = net(input)

input = input.numpy()
target = target.numpy()
output = output.detach().numpy()

plt.subplot(1, 4, 1)
plt.imshow(np.flip(input[0, 1, ...], axis=-2), cmap="magma")
plt.title("Input")

plt.subplot(1, 4, 2)
plt.imshow(np.flip(output[0, 1, ...], axis=-2), cmap="magma")
plt.title("Output")

plt.subplot(1, 4, 3)
plt.imshow(np.flip(target[0, 1, ...], axis=-2), cmap="magma")
plt.title("Target")

diff = target[0, 1, ...] - output[0, 1, ...]
plt.subplot(1, 4, 4)
plt.imshow(np.flip(diff, axis=-2), cmap="gray")
plt.title("Difference")

plt.show()

# Comprehensive example: solver-in-the-loop

The following example shows how to use
* the PBDLConstantBatchSampler
* PyTorch-PhiFlow tensor conversion

In [ ]:
from pbdl.torch.phi.loader import *
from examples.ks.ks_networks import ConvResNet1D
from examples.ks.ks_solver import DifferentiableKS

# training parameters
BATCH_SIZE = 16
LR = 1e-4
EPOCHS = 4

# solver parameters
RES = 48
TIMESTEP = 0.5
DOMAIN_SIZE_BASE = 8
PREDHORZ = 5

device = "cuda:0" if torch.cuda.is_available() else "cpu"

diff_ks = DifferentiableKS(resolution=RES, dt=TIMESTEP)

dataset = Dataset(
    "ks-dataset",
    time_steps=PREDHORZ,
    step_size=20,
    intermediate_time_steps=True,
    normalize=False,
)

batch_sampler = ConstantBatchSampler(dataset, BATCH_SIZE, group_constants=[0]) # group after first constant
dataloader = Dataloader(dataset, batch_sampler=batch_sampler)

net = ConvResNet1D(16, 3, device=device)
optimizer = torch.optim.Adam(net.parameters(), lr=LR)
loss = torch.nn.MSELoss()

Notice that the dataset is initialized with the `intermediate_time_steps` flag. The dataset now not only supplies the initial and the target frame, but also all frames in between.

We also use the PBDLConstantBatchSampler to ensure that all samples in a batch have the same constants. This is necessary because for each batch only one domain size can be passed to the solver function. You can use `group_constants` to specify which constants should be considered for grouping into batches.

In [ ]:
for epoch in range(EPOCHS):
    for i, (input, targets) in enumerate(dataloader):

        input = input.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        domain_size = input[0][1][0].item()

        inputs = [input]
        outputs = []

        for _ in range(PREDHORZ):
            output_solver = diff_ks.etd1(
                dataset.to_phiflow(inputs[-1]), DOMAIN_SIZE_BASE * domain_size
            )

            correction = diff_ks.dt * net(inputs[-1])
            output_combined = dataset.from_phiflow(output_solver) + correction

            outputs.append(output_combined)
            inputs.append(dataset.cat_constants(outputs[-1], inputs[0]))

        outputs = torch.stack(outputs, axis=1)

        loss_value = loss(outputs, targets)
        loss_value.backward()
        optimizer.step()

    print(f"epoch { epoch }, loss {(loss_value.item()*10000.) :.3f}")

When a solver is included in the training process, it is necessary to convert between PyTorch tensors and solver tensors and possibly add or remove constant layers. For this purpose, the PBDLDataset class provides the auxiliary methods `to_phiflow`, `from_phiflow` and `cat_constants` to convert between tensor types and to add constant layers to the solver output. `cat_constants` function takes a reference tensor as input, from which it copies the constant layers.

In [ ]:
input, targets = next(iter(dataloader))

domain_size = input[0][1][0].item()

inputs = [input]
outputs = []

for _ in range(PREDHORZ):
    output_solver = diff_ks.etd1(
        dataset.to_phiflow(inputs[-1]), DOMAIN_SIZE_BASE * domain_size
    )
    output_combined = dataset.from_phiflow(output_solver) + diff_ks.dt * net(inputs[-1])

    outputs.append(output_combined)
    inputs.append(dataset.cat_constants(outputs[-1], inputs[0]))

outputs = torch.stack(outputs, axis=1)

input = inputs[0][0][0:1, ...].detach().numpy()
output = outputs[0][-1].detach().numpy()
target = targets[0][-1]

plt.subplot(4, 1, 1)
plt.imshow(input, cmap="magma", aspect=1)
plt.title("input")

plt.subplot(4, 1, 2)
plt.imshow(output, cmap="magma", aspect=1)
plt.title("output")

plt.subplot(4, 1, 3)
plt.imshow(target, cmap="magma", aspect=1)
plt.title("target")

diff = target - output
plt.subplot(4, 1, 4)
plt.imshow(diff, cmap="gray", aspect=1)
plt.title("target-output difference")

plt.show()

# Dataset files

A `.hdf5`-file is just a hierarchically structured collection of arrays (see [official documentation](https://docs.h5py.org/en/stable/quick.html)). For pbdl datasets, the hierarchy is very flat &ndash; there is just one group `sim/` containing all simulations (numpy arrays). These arrays are named `sim` concatenated with an incremental index. The `.hdf5` file format supports attaching metadata to arrays; for pbdl datasets this is used to store the constants.

Each simulation array has the shape `(frame, field, spatial dims...)`. The attached constant array has the shape `(const)` for single-file datasets and `[const]` for partitioned datasets. The following code shows how to create the dataset file:

In [ ]:
import numpy as np
import h5py

sim = np.zeros((1000, 3, 64, 32))
const = np.zeros((2))

with h5py.File("mydataset.hdf5", "w") as f:
    dset = f.create_dataset("sims/sim0", data=sim)
    dset.attrs["const"] = const

    # you can add more simulation array 'sims/sim1', 'sims/sim2', ...

# Local datasets

In the following example we will add a local dataset to the index, so it can be used with PBDLDataset. First we need a dataset file, which we download from LRZ:

In [ ]:
import os
import urllib

os.makedirs(os.path.dirname("./local_datasets/"), exist_ok=True)

urllib.request.urlretrieve(
    "https://syncandshare.lrz.de/dl/fiFpt1oyzXW8J4uvg43JF8/transonic_cylinder_flow_tiny.hdf5",
    "./local_datasets/transonic_cylinder_flow_tiny.hdf",
)


PBDLDataset looks for datasets in two places: in the global index on the server and (if it exists) in the local index. The local index is a JSON file named `datasets.json` and located in the working directory. The following code creates such a file:

In [ ]:
import json

data = {
    "local-transonic-cylinder-flow": {
        "path": "./local_datasets/transonic_cylinder_flow_tiny.hdf5",
        "fields": "VVdp",
        "field_desc" : ["velocity x", "velocity y", "density", "pressure"],
        "const_desc" : ["mach number"]
    }
}

with open("datasets.json", "w") as file:
    json.dump(data, file, indent=4)

The local index file must have a specific structure: The path to your local `.npz`-file is specified by `path`. If the extension `.npz` is omitted the dataset is interpreted as a partitioned dataset. In this case, it is necessary the specify an additional attribute `num_part` for the number of partitions. `fields` contains information about the type of physical field in the form of a string, e.g. VVdp (velocity x, velocity y, density, pressure). Consecutive identical letters indicate that a physical field consists of several indices (vector field). This information affects how normalization is applied: For vector fields, the vector norm is applied first before the standard deviation is calculated.

Now that the local dataset is registered, we can use it with PBDLDataset:

In [ ]:
import importlib
import pbdl.loader

# package must be reloaded (not necessary in isolated code)
importlib.reload(pbdl.loader)

from pbdl.loader import *

dataset = Dataset(
    "local-transonic-cylinder-flow",
    time_steps=10,
    normalize=True
)